### Functions:
* [Audio splitter](#split_cell) -  Function to split audio files into smaller, even-sized files
* [CSV export](#csv_cell) -  Function to process audio and output csv files to save time when loading dataset in the future

<a id='split_cell'></a>

In [2]:
from pydub import AudioSegment
from os import listdir, makedirs

C:\Users\lukoz\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
def file_splitter(file_path, save_path, split_num = 5):
    """
    Function to split audio files into smaller, even files
    Parameters:
        file_path - file load directory
        save_path - file save directory
        split_num - number of smaller files to be made
    """
    print('\n\t Start splitting \n')
    file_list = listdir(file_path)
    
    # Creating save directories (if they don't already exist)
    try:
        makedirs(save_path)
    except:
        pass
    
    # Audio split
    for idx, file in enumerate(file_list):
        print('File %d out of %d' % (idx, len(file_list)))
        audio = AudioSegment.from_wav(file_path+file) # Load file
        file_duration = audio.duration_seconds * 1000  # Get duration of file in miliseconds
        cut_duration = file_duration/split_num # Calculate duration of new files
        for i in range(0,5):
            t1 = cut_duration * i # Starting time
            t2 = cut_duration * (i+1) # Ending time
            cut_audio = audio[t1:t2]
            path = save_path+file[0:-4]+ '_' + str(i)+ '.wav' # Dynamic file making
            cut_audio.export(path, format="wav") # Audio export

In [3]:
# Using the function
# file_splitter('./26-29_09_2017_KCL/ReadText/HC/', './After Splitting/ReadText/HC/')
# file_splitter('./26-29_09_2017_KCL/ReadText/PD/', './After Splitting/ReadText/PD/')
# file_splitter('./26-29_09_2017_KCL/SpontaneousDialogue/HC/', './After Splitting/SpontaneousDialogue/HC/')
# file_splitter('./26-29_09_2017_KCL/SpontaneousDialogue/PD/', './After Splitting/SpontaneousDialogue/PD/')

<a id='csv_cell'></a>

In [4]:
import pandas as pd
import opensmile

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [5]:
def csv_export(file_path, save_path):
    """
    Function to split audio files into smaller, even files
    Parameters:
        file_path - file load directory (Spontaneus Dialogue or Read Text)
        save_path - file save directory
    """
    for i in range(0,3):
        print('Processing %d feature set out of 3' % (i+1))
        feature_set = ''
        if i == 0:
            feature_set = 'GeMAPSv01b'
            smile = opensmile.Smile(
                feature_set=opensmile.FeatureSet.GeMAPSv01b,
                feature_level=opensmile.FeatureLevel.Functionals)
        elif i == 1:
            feature_set = 'eGeMAPSv01b'
            smile = opensmile.Smile(
                feature_set=opensmile.FeatureSet.eGeMAPSv01b,
                feature_level=opensmile.FeatureLevel.Functionals)
        elif i == 2:
            feature_set = 'ComParE2016'
            smile = opensmile.Smile(
                feature_set=opensmile.FeatureSet.ComParE_2016,
                feature_level=opensmile.FeatureLevel.Functionals)
            
        hc_list = smile.process_folder(file_path+'HC/')
        pd_list = smile.process_folder(file_path+'PD/')
        hc_list['targetValue'] = 0 # 0 for healthy patients
        pd_list['targetValue'] = 1 # 1 for PD patients
        full_list = pd_list.append(hc_list)
        full_list.reset_index(drop=True, inplace=True)
        path = save_path+'_' + feature_set + '.csv'
        full_list.to_csv(path)

In [6]:
# Using the function
# Split files
# csv_export('./After Splitting/ReadText/', './ReadText')
# csv_export('./After Splitting/SpontaneousDialogue/', './SpontaneousDialogue')

# Whole files
csv_export('./26-29_09_2017_KCL/ReadText/', './WholeReadText')
csv_export('./26-29_09_2017_KCL/SpontaneousDialogue/', './WholeSpontaneousDialogue')

Processing 1 feature set out of 3
Processing 2 feature set out of 3
Processing 3 feature set out of 3
Processing 1 feature set out of 3
Processing 2 feature set out of 3
Processing 3 feature set out of 3


In [8]:
# Testing the csvs
df = pd.read_csv('./csvs/ReadText_GeMAPSv01b.csv')
df.drop('Unnamed: 0', axis=1, inplace = True)
df.head()

,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,...,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,targetValue
0,26.725595,0.141266,24.359055,26.660196,29.649887,5.290832,384.09973,508.24286,46.179512,47.26151,...,25.040714,-0.013399,-0.017325,1.904762,1.304071,0.245366,0.180732,0.514250,1.139605,1
1,25.262962,0.207004,23.214577,25.123089,27.338364,4.123787,483.00836,816.43680,96.939090,145.44917,...,12.328325,-0.011125,-0.009469,3.047619,2.480916,0.252179,0.205948,0.133553,0.239806,1
2,25.031317,0.188289,23.097178,24.673878,26.626678,3.529501,470.76166,747.27450,74.679344,111.53612,...,15.881686,-0.011384,-0.011833,3.047619,2.258270,0.242535,0.185630,0.173151,0.240598,1
3,24.820858,0.192796,22.523014,24.346283,26.893549,4.370535,340.03217,420.03793,70.173700,70.23804,...,19.071331,-0.015491,-0.014352,3.047619,2.099905,0.255000,0.220909,0.193824,0.350567,1
4,24.961870,0.150287,22.891483,24.886480,26.944284,4.052801,366.44770,408.86160,79.374160,110.23552,...,22.351864,-0.013313,-0.016072,3.047619,1.875994,0.240678,0.197217,0.271017,0.446328,1


In [9]:
df = pd.read_csv('./csvs/ReadText_eGeMAPSv01b.csv')
df.drop('Unnamed: 0', axis=1, inplace = True)
df.head()

,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,...,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,targetValue
0,26.725595,0.141266,24.359055,26.660196,29.649887,5.290832,384.09973,508.24286,46.179512,47.26151,...,-0.017325,0.055772,1.904762,1.304071,0.245366,0.180732,0.514250,1.139605,-34.632526,1
1,25.262962,0.207004,23.214577,25.123089,27.338364,4.123787,483.00836,816.43680,96.939090,145.44917,...,-0.009469,0.109873,3.047619,2.480916,0.252179,0.205948,0.133553,0.239806,-35.001320,1
2,25.031317,0.188289,23.097178,24.673878,26.626678,3.529501,470.76166,747.27450,74.679344,111.53612,...,-0.011833,0.083195,3.047619,2.258270,0.242535,0.185630,0.173151,0.240598,-36.764767,1
3,24.820858,0.192796,22.523014,24.346283,26.893549,4.370535,340.03217,420.03793,70.173700,70.23804,...,-0.014352,0.068798,3.047619,2.099905,0.255000,0.220909,0.193824,0.350567,-36.190100,1
4,24.961870,0.150287,22.891483,24.886480,26.944284,4.052801,366.44770,408.86160,79.374160,110.23552,...,-0.016072,0.046315,3.047619,1.875994,0.240678,0.197217,0.271017,0.446328,-38.731415,1


In [10]:
df = pd.read_csv('./csvs/ReadText_ComParE2016.csv')
df.drop('Unnamed: 0', axis=1, inplace = True)
df.head()

,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,audspec_lengthL1norm_sma_percentile1.0,...,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope,targetValue
0,2.117525,0.559796,0.069338,0.046711,0.086149,0.465465,0.039438,0.379316,0.418754,0.040217,...,0.633474,2.114188,2.116042,-19.999958,0.592081,102.696670,48.370613,98.58104,48.153404,1
1,2.062647,0.168257,0.965649,0.167934,0.327994,0.517738,0.160061,0.189744,0.349805,0.048616,...,0.510338,2.482564,2.482743,-20.000000,0.615493,103.480125,53.692764,105.62107,54.380642,1
2,1.611995,0.698791,0.613868,0.122889,0.285024,0.444780,0.162135,0.159756,0.321891,0.045012,...,0.590135,2.402636,2.404758,-19.999954,0.499842,105.112070,54.212250,107.21883,57.699140,1
3,1.469307,0.047710,0.930025,0.090848,0.238185,0.404335,0.147337,0.166150,0.313487,0.043727,...,0.494766,2.262606,2.261649,20.000000,0.560532,105.850040,55.175697,102.49456,51.597220,1
4,1.066134,0.399491,0.917939,0.056036,0.177332,0.343132,0.121295,0.165800,0.287096,0.042189,...,0.536929,2.214892,2.215831,-20.000000,0.631352,104.949750,56.932274,100.86527,52.327354,1


In [37]:
def data_joining(path):
    path_end1 = '_GeMAPSv01b'
    path_end2 = '_eGeMAPSv01b' # contains entire GeMAPS base so 2 = 1 & 2 AND 2 & 3 = 1 & 2 & 3
    path_end3 = '_ComParE2016'
    df1 = pd.read_csv(path+path_end1+'.csv')
    df1.drop('Unnamed: 0', axis=1, inplace = True)
    df2 = pd.read_csv(path+path_end2+'.csv')
    df2.drop('Unnamed: 0', axis=1, inplace = True)
    df3 = pd.read_csv(path+path_end3+'.csv')
    df3.drop('Unnamed: 0', axis=1, inplace = True)

    # 1 & 3
    df1_3 = df1.copy()
    count = 0
    for col in df3.columns:
        if col not in df1.columns:
            df1_3[col] = df3[col]

    print('Starting length: %d; Ending length: %d; Columns added: %d' % 
          (len(df1.columns),len(df1_3.columns), len(df1_3.columns) - len(df1.columns) ))
    df1_3.to_csv(path+path_end1+path_end3+'.csv')

    # 2 & 3
    df2_3 = df2.copy()
    count = 0
    for col in df3.columns:
        if col not in df2.columns:
            df2_3[col] = df3[col]

    print('Starting length: %d; Ending length: %d; Columns added: %d' % 
          (len(df2.columns),len(df2_3.columns), len(df2_3.columns) - len(df2.columns) ))
    df2_3.to_csv(path+path_end2+path_end3+'.csv')


In [42]:
path = './csvs/ReadText'
data_joining(path)
path = './csvs/SpontaneousDialogue'
data_joining(path)

89
Starting length: 63; Ending length: 6436; Columns added: 6373
Starting length: 89; Ending length: 6462; Columns added: 6373
89
Starting length: 63; Ending length: 6436; Columns added: 6373
Starting length: 89; Ending length: 6462; Columns added: 6373


In [23]:
df = pd.read_csv('./csvs/ReadText_GeMAPSv01b_eGeMAPSv01b.csv')
df.drop('Unnamed: 0', axis=1, inplace = True)
df.head()

,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,...,mfcc1V_sma3nz_amean,mfcc1V_sma3nz_stddevNorm,mfcc2V_sma3nz_amean,mfcc2V_sma3nz_stddevNorm,mfcc3V_sma3nz_amean,mfcc3V_sma3nz_stddevNorm,mfcc4V_sma3nz_amean,mfcc4V_sma3nz_stddevNorm,spectralFluxUV_sma3nz_amean,equivalentSoundLevel_dBp
0,26.725595,0.141266,24.359055,26.660196,29.649887,5.290832,384.09973,508.24286,46.179512,47.26151,...,26.154663,0.447678,17.169110,0.817369,12.094230,1.338822,3.344343,4.906656,0.055772,-34.632526
1,25.262962,0.207004,23.214577,25.123089,27.338364,4.123787,483.00836,816.43680,96.939090,145.44917,...,24.365824,0.571365,24.020885,0.575929,15.218651,1.020170,4.083021,3.707661,0.109873,-35.001320
2,25.031317,0.188289,23.097178,24.673878,26.626678,3.529501,470.76166,747.27450,74.679344,111.53612,...,25.700712,0.538119,26.022100,0.542918,11.719027,1.291158,1.306171,13.199849,0.083195,-36.764767
3,24.820858,0.192796,22.523014,24.346283,26.893549,4.370535,340.03217,420.03793,70.173700,70.23804,...,27.202541,0.445577,26.396523,0.502365,13.138992,1.090150,4.332277,3.484381,0.068798,-36.190100
4,24.961870,0.150287,22.891483,24.886480,26.944284,4.052801,366.44770,408.86160,79.374160,110.23552,...,28.212004,0.426739,26.121157,0.483157,13.538903,1.050910,2.660480,6.301862,0.046315,-38.731415


In [41]:
display(df)

,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope,...,mfcc1V_sma3nz_amean,mfcc1V_sma3nz_stddevNorm,mfcc2V_sma3nz_amean,mfcc2V_sma3nz_stddevNorm,mfcc3V_sma3nz_amean,mfcc3V_sma3nz_stddevNorm,mfcc4V_sma3nz_amean,mfcc4V_sma3nz_stddevNorm,spectralFluxUV_sma3nz_amean,equivalentSoundLevel_dBp
0,26.725595,0.141266,24.359055,26.660196,29.649887,5.290832,384.09973,508.24286,46.179512,47.26151,...,26.154663,0.447678,17.169110,0.817369,12.094230,1.338822,3.344343,4.906656,0.055772,-34.632526
1,25.262962,0.207004,23.214577,25.123089,27.338364,4.123787,483.00836,816.43680,96.939090,145.44917,...,24.365824,0.571365,24.020885,0.575929,15.218651,1.020170,4.083021,3.707661,0.109873,-35.001320
2,25.031317,0.188289,23.097178,24.673878,26.626678,3.529501,470.76166,747.27450,74.679344,111.53612,...,25.700712,0.538119,26.022100,0.542918,11.719027,1.291158,1.306171,13.199849,0.083195,-36.764767
3,24.820858,0.192796,22.523014,24.346283,26.893549,4.370535,340.03217,420.03793,70.173700,70.23804,...,27.202541,0.445577,26.396523,0.502365,13.138992,1.090150,4.332277,3.484381,0.068798,-36.190100
4,24.961870,0.150287,22.891483,24.886480,26.944284,4.052801,366.44770,408.86160,79.374160,110.23552,...,28.212004,0.426739,26.121157,0.483157,13.538903,1.050910,2.660480,6.301862,0.046315,-38.731415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,30.362846,0.505434,14.085036,27.140245,48.391120,34.306084,347.71835,289.68802,209.080180,389.97818,...,20.290602,0.297014,17.373842,0.526587,13.638556,0.723958,-11.606460,-0.788081,0.019104,-50.082820
181,26.755062,0.106312,25.382618,27.005789,28.441238,3.058620,284.92865,434.64505,81.478010,129.11357,...,30.961820,0.385259,15.938849,0.943050,18.482298,0.856100,1.428049,8.443448,0.045685,-39.251330
182,26.620838,0.110749,25.550850,26.981264,28.341480,2.790630,303.14627,340.30444,110.502250,257.78677,...,31.434502,0.391707,17.300264,0.791094,18.599140,0.759724,0.963141,11.315509,0.047636,-39.249092
183,27.143133,0.117889,25.936958,27.263157,29.148674,3.211716,300.22107,367.44614,89.787720,105.24958,...,33.510920,0.358981,14.420041,1.080094,18.096697,0.781541,-1.359708,-9.468228,0.040854,-38.438183
